In [1]:
import sys
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import time
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from os import listdir
from numpy import asarray
from numpy import save
from tensorflow.keras.preprocessing.image import load_img
import io
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image

In [3]:
# resizing the images
from PIL import Image
import os

# Set the desired size
size = (200, 200)

# Loop through the directories and resize the images
for folder in ['dogs_vs_squirrel/test/dogs/', 'dogs_vs_squirrel/test/squirrel/']:
    for filename in os.listdir(folder):
        image_path = os.path.join(folder, filename)
        with Image.open(image_path) as image:
            resized_image = image.resize(size)
            resized_image.save(image_path)

In [2]:
images = []

In [5]:
import os


folder_path = 'dogs_vs_squirrel/test/squirrel/'
# Loop over all the files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is an image file
    if filename.endswith('.png'):
        # Open the image file using PIL
        image = Image.open(os.path.join(folder_path, filename))
        # Append the image to the list
        images.append(image)

In [6]:
print(len(images))

40


In [7]:
def plot_to_image(figure):
    buf = io.BytesIO()
    pyplot.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    pyplot.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def image_grid(predictions, images):
    figure = pyplot.figure(figsize=(10, 10))
    for i in range(40):
            # Start next subplot.
        pyplot.subplot(5, 8, i + 1, title= 'Dogs' if predictions[i] < 0.5 else 'Squirrel')
        pyplot.xticks([])
        pyplot.yticks([])
        pyplot.grid(False)
        pyplot.imshow(images[i])

    return figure

def load_image(filename):
     # load the image
    img = load_img(filename, target_size=(200, 200))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 200, 200, 3)
    # center pixel data
    img = img.astype('float32')
    img = img - [123.68, 116.779, 103.939]
    return img

In [8]:
test_images = []

In [10]:
# create an array of test images

for i in range(81, 101):
    # load the image
    img = load_image('dogs_vs_squirrel/test/dogs/' +'dogs_' + str(i) + '.png')
    # store loaded image
    test_images.append(img)

In [11]:
print(len(test_images))

40


In [12]:
# VGG 1 Block
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))
 # compile model
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
vgg1_callback = keras.callbacks.TensorBoard(log_dir="logs/vgg1_callback_dir", update_freq = 'batch')

In [13]:

# plot diagnostic learning curves
def summarize_diagnostics(history, text):
     # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    pyplot.savefig(text + 'plot.png')
    pyplot.close()

In [16]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
# prepare iterators
train_it = datagen.flow_from_directory('dogs_vs_squirrel/train/', class_mode='binary', batch_size=80, target_size=(200, 200))
test_it = datagen.flow_from_directory('dogs_vs_squirrel/test/', class_mode='binary', batch_size=40, target_size=(200, 200))

# fit model
start_time = time.time()
history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=18, callbacks=[vgg1_callback], verbose=2)
training_time = time.time() - start_time

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/18
2/2 - 4s - loss: 1.7817 - accuracy: 0.4500 - val_loss: 1.0808 - val_accuracy: 0.5000 - 4s/epoch - 2s/step
Epoch 2/18
2/2 - 3s - loss: 1.0586 - accuracy: 0.5000 - val_loss: 0.6765 - val_accuracy: 0.5500 - 3s/epoch - 1s/step
Epoch 3/18
2/2 - 3s - loss: 0.6859 - accuracy: 0.5437 - val_loss: 0.6919 - val_accuracy: 0.5000 - 3s/epoch - 1s/step
Epoch 4/18
2/2 - 3s - loss: 0.6831 - accuracy: 0.5125 - val_loss: 0.6827 - val_accuracy: 0.5250 - 3s/epoch - 1s/step
Epoch 5/18
2/2 - 3s - loss: 0.6843 - accuracy: 0.5250 - val_loss: 0.6880 - val_accuracy: 0.5000 - 3s/epoch - 1s/step
Epoch 6/18
2/2 - 3s - loss: 0.6534 - accuracy: 0.6750 - val_loss: 0.6704 - val_accuracy: 0.6000 - 3s/epoch - 1s/step
Epoch 7/18
2/2 - 3s - loss: 0.6297 - accuracy: 0.7375 - val_loss: 0.6621 - val_accuracy: 0.5750 - 3s/epoch - 1s/step
Epoch 8/18
2/2 - 3s - loss: 0.6326 - accuracy: 0.6562 - val_loss: 0.7180 - val_accuracy: 0.5500 - 3s

In [17]:
# Prepare the plot
file_writer = tf.summary.create_file_writer("logs/vgg1_callback_dir/test_images")
predictions = []
for i in test_images:
    predictions.append(model.predict(i)[0][0])

figure = image_grid(predictions, images)
#pyplot.plot(figure)
# Convert to image and log
with file_writer.as_default():
    tf.summary.image("Test Images", plot_to_image(figure), step=0)

# Calculate training and testing metrics
train_loss, train_accuracy = model.evaluate(train_it, steps=len(train_it), verbose=0)
test_loss, test_accuracy = model.evaluate(test_it, steps=len(test_it), verbose=0)

# Count the number of model parameters
num_params = model.count_params()

print('> %.3f' % (train_accuracy * 100.0))
# learning curves
summarize_diagnostics(history, 'VGG 1 Block ')

# Append the Block model metrics to the DataFrame
df.loc[0] = ["VGG (1 blocks)", training_time, train_loss, train_accuracy, test_accuracy, num_params]

1/1 [==============================] - 0s 33ms/step
> 70.625


In [19]:
# VGG 3 Block
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))
# compile model
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
vgg3_callback = keras.callbacks.TensorBoard(log_dir="logs/vgg3_callback_dir", update_freq = 'batch')


In [20]:
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0)
# prepare iterators
train_it = datagen.flow_from_directory('dogs_vs_squirrel/train/', class_mode='binary', batch_size=80, target_size=(200, 200))
test_it = datagen.flow_from_directory('dogs_vs_squirrel/test/', class_mode='binary', batch_size=40, target_size=(200, 200))
# fit model
start_time = time.time()
history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=18, callbacks=[vgg3_callback], verbose=2)
training_time = time.time() - start_time

# Prepare the plot
file_writer = tf.summary.create_file_writer("logs/vgg3_callback_dir/test_images")
predictions = []
for i in test_images:
    predictions.append(model.predict(i)[0][0])

figure = image_grid(predictions, images)
#pyplot.plot(figure)
# Convert to image and log
with file_writer.as_default():
    tf.summary.image("Test Images", plot_to_image(figure), step=0)
    
# Calculate training and testing metrics
train_loss, train_accuracy = model.evaluate(train_it, steps=len(train_it), verbose=0)
test_loss, test_accuracy = model.evaluate(test_it, steps=len(test_it), verbose=0)

# Count the number of model parameters
num_params = model.count_params()

print('> %.3f' % (train_accuracy * 100.0))
# learning curves
summarize_diagnostics(history, 'VGG 3 Block ')
# Append the Block model metrics to the DataFrame
df.loc[1] = ["VGG (3 blocks)", training_time, train_loss, train_accuracy, test_accuracy, num_params]


Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/18
2/2 - 6s - loss: 2.4032 - accuracy: 0.5000 - val_loss: 1.3088 - val_accuracy: 0.5000 - 6s/epoch - 3s/step
Epoch 2/18
2/2 - 6s - loss: 1.1973 - accuracy: 0.4875 - val_loss: 1.0029 - val_accuracy: 0.5000 - 6s/epoch - 3s/step
Epoch 3/18
2/2 - 5s - loss: 0.9334 - accuracy: 0.5000 - val_loss: 0.7127 - val_accuracy: 0.5000 - 5s/epoch - 3s/step
Epoch 4/18
2/2 - 5s - loss: 0.7022 - accuracy: 0.5000 - val_loss: 0.6865 - val_accuracy: 0.5250 - 5s/epoch - 3s/step
Epoch 5/18
2/2 - 6s - loss: 0.6922 - accuracy: 0.4938 - val_loss: 0.6863 - val_accuracy: 0.6750 - 6s/epoch - 3s/step
Epoch 6/18
2/2 - 6s - loss: 0.6897 - accuracy: 0.5188 - val_loss: 0.6845 - val_accuracy: 0.5750 - 6s/epoch - 3s/step
Epoch 7/18
2/2 - 6s - loss: 0.6891 - accuracy: 0.5312 - val_loss: 0.6833 - val_accuracy: 0.5250 - 6s/epoch - 3s/step
Epoch 8/18
2/2 - 6s - loss: 0.6876 - accuracy: 0.5312 - val_loss: 0.6824 - val_accuracy: 0.5500 - 6s

In [26]:
# VGG 3 Block
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))
# compile model
opt = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
vgg3data_callback = keras.callbacks.TensorBoard(log_dir="logs/vgg3data_callback_dir", update_freq = 'batch')


In [27]:
#VGG 3 blocks with data augmentation
# create data generators
#vgg3data_callback = keras.callbacks.TensorBoard(log_dir="logs/vgg3data_callback_dir", update_freq='batch')
train_datagen = ImageDataGenerator(rescale=1.0/255.0, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
# prepare iterators
train_it = train_datagen.flow_from_directory('dogs_vs_squirrel/train/', class_mode='binary', batch_size=80, target_size=(200, 200))
test_it = test_datagen.flow_from_directory('dogs_vs_squirrel/test/', class_mode='binary', batch_size=40, target_size=(200, 200))
start_time = time.time()
history = model.fit(train_it, steps_per_epoch=len(train_it), validation_data=test_it, validation_steps=len(test_it), epochs=18, callbacks=[vgg3data_callback], verbose=2)
training_time = time.time() - start_time

# Prepare the plot
file_writer = tf.summary.create_file_writer("logs/vgg3data_callback_dir/test_images")
predictions = []
for i in test_images:
    predictions.append(model.predict(i)[0][0])

figure = image_grid(predictions, images)
#pyplot.plot(figure)
# Convert to image and log
with file_writer.as_default():
    tf.summary.image("Test Images", plot_to_image(figure), step=0)
    
# Calculate training and testing metrics
train_loss, train_accuracy = model.evaluate(train_it, steps=len(train_it), verbose=0)
test_loss, test_accuracy = model.evaluate(test_it, steps=len(test_it), verbose=0)

# Count the number of model parameters
num_params = model.count_params()

print('> %.3f' % (train_accuracy * 100.0))

# learning curves
summarize_diagnostics(history, 'VGG 3 Block w augmentation ')
# Append the Block model metrics to the DataFrame
df.loc[2] = ["VGG (3 blocks w augmentation)", training_time, train_loss, train_accuracy, test_accuracy, num_params]

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/18
2/2 - 7s - loss: 1.3300 - accuracy: 0.4625 - val_loss: 0.8800 - val_accuracy: 0.5000 - 7s/epoch - 3s/step
Epoch 2/18
2/2 - 6s - loss: 0.8402 - accuracy: 0.5000 - val_loss: 0.7029 - val_accuracy: 0.5500 - 6s/epoch - 3s/step
Epoch 3/18
2/2 - 6s - loss: 0.7497 - accuracy: 0.5188 - val_loss: 0.7300 - val_accuracy: 0.5000 - 6s/epoch - 3s/step
Epoch 4/18
2/2 - 6s - loss: 0.6881 - accuracy: 0.5312 - val_loss: 0.7317 - val_accuracy: 0.5000 - 6s/epoch - 3s/step
Epoch 5/18
2/2 - 6s - loss: 0.7249 - accuracy: 0.5000 - val_loss: 0.6913 - val_accuracy: 0.5000 - 6s/epoch - 3s/step
Epoch 6/18
2/2 - 6s - loss: 0.6915 - accuracy: 0.5125 - val_loss: 0.7041 - val_accuracy: 0.5000 - 6s/epoch - 3s/step
Epoch 7/18
2/2 - 7s - loss: 0.7143 - accuracy: 0.5000 - val_loss: 0.7036 - val_accuracy: 0.5000 - 7s/epoch - 3s/step
Epoch 8/18
2/2 - 7s - loss: 0.6875 - accuracy: 0.5375 - val_loss: 0.6812 - val_accuracy: 0.5000 - 7s

In [32]:
from keras.applications.vgg16 import VGG16
from keras.models import Model

# transfer learning using vgg16

model1 = VGG16(include_top=False, input_shape=(200, 200, 3))
# mark loaded layers as not trainable
for layer in model1.layers:
    layer.trainable = False
 # add new classifier layers
flat1 = Flatten()(model1.layers[-1].output)
class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
output = Dense(1, activation='sigmoid')(class1)
# define new model
model1 = Model(inputs=model1.inputs, outputs=output)
# compile model
opt = SGD(learning_rate=0.001, momentum=0.9)
model1.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
vgg16_callback = keras.callbacks.TensorBoard(log_dir="logs/vgg16_callback_dir", update_freq='batch')


In [33]:
datagen = ImageDataGenerator(featurewise_center=True)
# specify imagenet mean values for centering
datagen.mean = [123.68, 116.779, 103.939]
# prepare iterator
train_it = datagen.flow_from_directory('dogs_vs_squirrel/train/', class_mode='binary', batch_size=80, target_size=(200, 200))
test_it = datagen.flow_from_directory('dogs_vs_squirrel/test/', class_mode='binary', batch_size=40, target_size=(200, 200))
 # fit model
start_time = time.time()
history = model1.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=18, callbacks=[vgg16_callback], verbose=2)
training_time = time.time() - start_time

# Prepare the plot
file_writer = tf.summary.create_file_writer("logs/vgg16_callback_dir/test_images")
predictions = []
for i in test_images:
    predictions.append(model.predict(i)[0][0])

figure = image_grid(predictions, images)
#pyplot.plot(figure)
# Convert to image and log
with file_writer.as_default():
    tf.summary.image("Test Images", plot_to_image(figure), step=0)
    
    
# Calculate training and testing metrics
train_loss, train_accuracy = model1.evaluate(train_it, steps=len(train_it), verbose=0)
test_loss, test_accuracy = model1.evaluate(test_it, steps=len(test_it), verbose=0)

# Count the number of model parameters
num_params = model1.count_params()

print('> %.3f' % (train_accuracy * 100.0))

# learning curves
summarize_diagnostics(history, 'VGG 16')
# Append the Block model metrics to the DataFrame
df.loc[3] = ["VGG 16", training_time, train_loss, train_accuracy, test_accuracy, num_params]

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


C:\Users\DELL\AppData\Local\Temp\ipykernel_15436\3302653080.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model1.fit_generator(train_it, steps_per_epoch=len(train_it),validation_data=test_it, validation_steps=len(test_it), epochs=18, callbacks=[vgg16_callback], verbose=2)


Epoch 1/18
2/2 - 21s - loss: 2.7290 - accuracy: 0.6875 - val_loss: 0.0647 - val_accuracy: 0.9750 - 21s/epoch - 10s/step
Epoch 2/18
2/2 - 24s - loss: 0.0172 - accuracy: 0.9937 - val_loss: 0.2803 - val_accuracy: 0.9500 - 24s/epoch - 12s/step
Epoch 3/18
2/2 - 29s - loss: 0.1621 - accuracy: 0.9875 - val_loss: 0.3904 - val_accuracy: 0.9500 - 29s/epoch - 14s/step
Epoch 4/18
2/2 - 26s - loss: 1.4652e-05 - accuracy: 1.0000 - val_loss: 0.3792 - val_accuracy: 0.9750 - 26s/epoch - 13s/step
Epoch 5/18
2/2 - 25s - loss: 1.0129e-09 - accuracy: 1.0000 - val_loss: 0.3942 - val_accuracy: 0.9750 - 25s/epoch - 13s/step
Epoch 6/18
2/2 - 25s - loss: 1.7008e-10 - accuracy: 1.0000 - val_loss: 0.3980 - val_accuracy: 0.9750 - 25s/epoch - 12s/step
Epoch 7/18
2/2 - 24s - loss: 1.6161e-10 - accuracy: 1.0000 - val_loss: 0.3991 - val_accuracy: 0.9750 - 24s/epoch - 12s/step
Epoch 8/18
2/2 - 25s - loss: 4.3087e-10 - accuracy: 1.0000 - val_loss: 0.4021 - val_accuracy: 0.9750 - 25s/epoch - 13s/step
Epoch 9/18
2/2 - 25s

In [47]:
import os
import shutil

folder_path = 'dogs_vs_squirrel/train/squirrel'

i = 1
for filename in os.listdir(folder_path):
    if filename.endswith('.png'):
        new_filename = 'squirrel_' + str(i) + '.png'
        src = os.path.join(folder_path, filename)
        dst = os.path.join(folder_path, new_filename)
        shutil.move(src, dst)
        i += 1

In [48]:
# load dogs vs cats dataset, reshape and save to a new file
# dogs -> 1
# squirrel -> 0
from os import listdir
from numpy import asarray
from numpy import save
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# define location of dataset
folder = 'train/'
photos, labels = list(), list()
# enumerate files in the directory
for file in listdir(folder):
    # determine class
    output = 0.0
    if file.startswith('dogs'):
        output = 1.0
    # load image
    photo = load_img(folder + file, target_size=(224, 224))
    # convert to numpy array
    photo = img_to_array(photo)
    # store
    photos.append(photo)
    labels.append(output)
    
# convert to a numpy arrays
photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)
# save the reshaped photos
save('dogs_vs_squirrel_photos.npy', photos)
save('dogs_vs_squirrel_labels.npy', labels)

(200, 224, 224, 3) (200,)


In [35]:
# seed random number generator
# organize dataset into a useful structure
from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random

seed(1)
dataset_home = 'dogs_vs_squirrel/'
# define ratio of pictures to use for validation
val_ratio = 0.2
# copy training dataset images into subdirectories
src_directory = 'train/'
for file in listdir(src_directory):
    src = src_directory + '/' + file
    dst_dir = 'train/'
    if random() < val_ratio:
        dst_dir = 'test/'
    if file.startswith('Daisy'):
        dst = dataset_home + dst_dir + 'Daisy/'  + file
        copyfile(src, dst)
    elif file.startswith('Sunflower'):
        dst = dataset_home + dst_dir + 'Sunflower/'  + file
        copyfile(src, dst)

In [14]:
import pandas as pd

# Create an empty DataFrame with the desired columns
df = pd.DataFrame(columns=["Model", "Training time", "Training loss", "Training accuracy", "Testing accuracy", "Number of model parameters"])


In [34]:
df

,Model,Training time,Training loss,Training accuracy,Testing accuracy,Number of model parameters
0,VGG (1 blocks),49.639443,5.849457e-01,0.70625,0.625,40961153
1,VGG (3 blocks),112.006978,6.356205e-01,0.72500,0.550,10333505
2,VGG (3 blocks w augmentation),120.805475,6.752859e-01,0.58750,0.650,10333505
3,VGG 16,463.923356,4.860777e-08,1.00000,0.975,17074241
